In [2]:
import glob
import dask.bag as db
import zipfile
import os
import re
import pickle
import json

1) Подсчитать количество слов с отзывами == 5.0

In [3]:
b = db.read_text('../data/reviews_Video_Games_5.json.gz').map(json.loads)

In [15]:
def clear_words(word):
    w = word.strip()
    return re.sub(r'[!?.,:;"()]', '', w.strip())

In [16]:
r5 = b.filter(lambda x: x['overall'] == 5.0).\
   map(lambda x: x['reviewText']).\
   str.lower().\
   str.split(' ').\
   flatten().\
   map(clear_words).\
   frequencies().compute()

In [17]:
len(r5)

361101

2) Для одного из файлов с отзывами из файлов. Подсчитать частоту (!) слов для отзывов с оценкой "5" и для отзывов с оценкой "2" и найти 50 распространенных слов, частота которых максимально увеличивается и уменьшается при переходе от плохих отзывов к хорошим.

In [19]:
r5 = b.filter(lambda x: x['overall'] == 5.0).\
   map(lambda x: x['reviewText']).\
   str.lower().\
   str.split(' ').\
   flatten().\
   map(clear_words).\
   frequencies().compute()

In [20]:
r2 = b.filter(lambda x: x['overall'] == 2.0).\
   map(lambda x: x['reviewText']).\
   str.lower().\
   str.split(' ').\
   flatten().\
   map(clear_words).\
   frequencies().compute()

In [21]:
r5.sort(key=lambda x: x[1], reverse=True)

In [22]:
s = 0
for i in r2:
    s += i[1]
print(s)

3384101


In [23]:
r = db.from_sequence([dict(r5), dict(r2)])

In [28]:
print(len(r5))
print(len(r2))

361101
99723


In [29]:
words = db.from_sequence(r2).map(lambda x: x[0])

In [30]:
def apply_pluck(word):
    r.pluck(word)

In [32]:
w = words.map(lambda word: (word, r.pluck(word)))

In [90]:
w = words.map(lambda word: (word, r.pluck(word))).map(lambda x: (x[0], abs(x[1][0]-x[1][1]))).compute()

TypeError: 'Bag' object does not support indexing

In [33]:
r2_d = dict(r2)
r5_d = dict(r5)

In [44]:
distances = {}
for k, v in r5_d.items():
    try:
        distances[k] = abs(r5_d[k] - r2_d.get(k))
    except:
        continue

sorted(distances.items(), reverse=True, key=lambda x: x[1])[0:50]

[('the', 946526),
 ('and', 526319),
 ('to', 444577),
 ('a', 438308),
 ('of', 356363),
 ('is', 334728),
 ('you', 324964),
 ('i', 306665),
 ('game', 283148),
 ('it', 268109),
 ('', 253583),
 ('this', 243615),
 ('in', 210346),
 ('that', 179782),
 ('for', 176474),
 ('are', 142863),
 ('with', 139751),
 ('on', 122737),
 ('have', 120639),
 ('but', 120051),
 ('as', 118662),
 ('can', 96892),
 ('your', 84023),
 ('games', 83692),
 ('be', 78825),
 ('my', 77676),
 ('not', 77604),
 ('one', 77005),
 ('like', 76995),
 ('was', 76116),
 ('all', 75484),
 ('so', 75354),
 ('if', 75202),
 ('or', 67853),
 ('play', 62907),
 ('great', 62781),
 ('more', 62238),
 ('get', 61381),
 ('will', 61041),
 ('from', 57107),
 ("it's", 56919),
 ('just', 56748),
 ('has', 55274),
 ('there', 54911),
 ('at', 53441),
 ('out', 51354),
 ('up', 49367),
 ('an', 49219),
 ('very', 48991),
 ('they', 48592)]